<a href="https://colab.research.google.com/github/kunal24bit/GAN-zero_to_hero/blob/main/GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
pip install -U keras-tuner

     |████████████████████████████████| 71kB 6.7MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=e5711e4d26c88c5daaf8c9f0dd1587054929bc0c3d718b0f93ecc64ce69a4e14
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=26ddad66546191bf131ec1d91f7d1668cddd11e8285a6de851c53e2ce064fff2
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


**Generative Adversarial Network**

GAN are generative models. They create new instances that resembles your training data.

GAN has two part:

1. Genreator
2. Discriminator

Where Generative models try to model how data is placed throghout the space while the other one try to draw boundaries in the data space.

GAN follows the alternating training process that is
1. Discriminator trains for one or more epochs
2. Generator trains for one or more epochs
3. Repeat step 1 and 2 to train Generator and Discriminator networks.

1. Discriminator tries to classify both real data and fake data from the generator.
2. Discriminator loss penalizes the discriminator for missclassifying a real instances as fake or fake instances as real.

3. Discriminator update its weight through Backpropagation.

4. genrator part of GAN learns to create fake data by incorporating feedback from the discriminator

**Loss Function**

The loss function which is used in original GAN paper is *minimax loss function*. In that function generator tries to minimize the following function 
while the discriminator tries to maximize the following function:

              E_x[log(D(x))]+E_z[log(1-D(G(z)))]


**Applications of GAN**

1. Generate Examples for Image Datasets
2. Generate Photographs of Human Faces
3. Generate Realistic Photographs
4. Generate Cartoon Characters

In [2]:
from tensorflow import keras
from keras.datasets import mnist

from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt
import sys
import numpy as np





In [4]:
class GAN():
  def __init__(self):
    self.img_rows = 28
    self.img_cols = 28
    self.channels = 1
    self.img_shape = (self.img_rows, self.img_cols, self.channels)
    self.latent_dim = 100

    optimizer = Adam(0.0002, 0.5)

    #Build and compile the discriminator
    self.discriminator  = self.build_discriminator()
    self.discriminator.compile(loss = 'binary_crossentropy', optimizer = optimizer, metrics = ['accuracy'])

    #Build the generator
    self.generator = self.build_generator()

    #The generator takes noise as input and generate imgs
    z = Input(shape=self.latent_dim,)

    img = self.generator(z)

    #For the combined model we will train generator only

    self.discriminator.trainable = False

    #The discriminator takes generated image as input and determines validity

    validity = self.discriminator(img)

    #The combined model(stacked generator and discriminator)
    #trains the generator to fool the discriminator

    self.combined = Model(z, validity)
    self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)
  

  def build_generator(self):

    model = Sequential()

    model.add(Dense(256, input_dim = self.latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(np.prod(self.img_shape), activation='tanh'))
    model.add(Reshape(self.img_shape))

    model.summary()

    noise = Input(shape=(self.latent_dim,))
    img = model(noise)

    return Model(noise, img)

  def build_discriminator(self):
     model = Sequential()

     model.add(Flatten(input_shape = self.img_shape))
     model.add(Dense(512))
     model.add(LeakyReLU(alpha=0.2))
     model.add(Dense(256))
     model.add(LeakyReLU(alpha=0.2))
     model.add(Dense(1, activation = 'sigmoid'))
     
     model.summary()

     img = Input(shape=self.img_shape)
     validity = model(img)

     return Model(img, validity)

  def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Generate a batch of new images
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Train the generator (to have the discriminator label samples as valid)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)

  def sample_images(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/%d.png" % epoch)
        plt.close()


if __name__ == '__main__':
    gan = GAN()
    gan.train(epochs=30000, batch_size=32, sample_interval=200)



    




























Streaming output truncated to the last 5000 lines.
25000 [D loss: 0.620734, acc.: 68.75%] [G loss: 1.063193]
25001 [D loss: 0.599973, acc.: 70.31%] [G loss: 1.025190]
25002 [D loss: 0.664935, acc.: 53.12%] [G loss: 1.135624]
25003 [D loss: 0.621018, acc.: 65.62%] [G loss: 0.967090]
25004 [D loss: 0.603668, acc.: 73.44%] [G loss: 1.130478]
25005 [D loss: 0.657751, acc.: 59.38%] [G loss: 1.041999]
25006 [D loss: 0.591521, acc.: 67.19%] [G loss: 0.919370]
25007 [D loss: 0.617478, acc.: 68.75%] [G loss: 0.910873]
25008 [D loss: 0.742372, acc.: 51.56%] [G loss: 1.009143]
25009 [D loss: 0.605677, acc.: 73.44%] [G loss: 1.047724]
25010 [D loss: 0.653740, acc.: 64.06%] [G loss: 1.066248]
25011 [D loss: 0.667785, acc.: 59.38%] [G loss: 0.991394]
25012 [D loss: 0.725810, acc.: 56.25%] [G loss: 0.948375]
25013 [D loss: 0.694439, acc.: 56.25%] [G loss: 1.058311]
25014 [D loss: 0.569382, acc.: 67.19%] [G loss: 1.061163]
25015 [D loss: 0.724267, acc.: 57.81%] [G loss: 0.989690]
25016 [D loss: 0.6815